from pyspark.sql import SparkSession
spark = SparkSession.builder\
.master("local")\
.appName("Data cleaning")\
.getOrCreate()

df = spark.read.format("csv").load('s3://mle7-data/train_numeric.csv')

In [1]:
# This row works only with SparkMagic
%%info

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1536915562292_0045,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


unknown magic command '%info'
UnknownMagic: unknown magic command '%info'



In [2]:
import os
import pyspark
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.amazonaws:aws-java-sdk-pom:1.10.34,org.apache.hadoop:hadoop-aws:2.7.2 pyspark-shell'
from pyspark.sql import SQLContext
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

In [3]:
filePath = "s3a://mle7-data/train_numeric.csv"
df = sqlContext.read.load(filePath,format="csv", inferSchema="true", header="true").cache()

### Data exploration
df.printSchema()

df.count()

#firstrows=df.take(5) # this one returns a list, not a dataframe!
firstrowsdf=df.limit(50)
firstrowsdf

# Print with pandas
import pandas as pd

#pd.DataFrame(firstrows, columns=df.columns).transpose()
firstrowsdf.toPandas().transpose()

In [5]:
#df.describe().toPandas().transpose()
#firstrowsdf.describe()

In [6]:
featureColumnNames=df.schema.names[1:969]
featureColumnNames

['L0_S0_F0', 'L0_S0_F2', 'L0_S0_F4', 'L0_S0_F6', 'L0_S0_F8', 'L0_S0_F10', 'L0_S0_F12', 'L0_S0_F14', 'L0_S0_F16', 'L0_S0_F18', 'L0_S0_F20', 'L0_S0_F22', 'L0_S1_F24', 'L0_S1_F28', 'L0_S2_F32', 'L0_S2_F36', 'L0_S2_F40', 'L0_S2_F44', 'L0_S2_F48', 'L0_S2_F52', 'L0_S2_F56', 'L0_S2_F60', 'L0_S2_F64', 'L0_S3_F68', 'L0_S3_F72', 'L0_S3_F76', 'L0_S3_F80', 'L0_S3_F84', 'L0_S3_F88', 'L0_S3_F92', 'L0_S3_F96', 'L0_S3_F100', 'L0_S4_F104', 'L0_S4_F109', 'L0_S5_F114', 'L0_S5_F116', 'L0_S6_F118', 'L0_S6_F122', 'L0_S6_F132', 'L0_S7_F136', 'L0_S7_F138', 'L0_S7_F142', 'L0_S8_F144', 'L0_S8_F146', 'L0_S8_F149', 'L0_S9_F155', 'L0_S9_F160', 'L0_S9_F165', 'L0_S9_F170', 'L0_S9_F175', 'L0_S9_F180', 'L0_S9_F185', 'L0_S9_F190', 'L0_S9_F195', 'L0_S9_F200', 'L0_S9_F205', 'L0_S9_F210', 'L0_S10_F219', 'L0_S10_F224', 'L0_S10_F229', 'L0_S10_F234', 'L0_S10_F239', 'L0_S10_F244', 'L0_S10_F249', 'L0_S10_F254', 'L0_S10_F259', 'L0_S10_F264', 'L0_S10_F269', 'L0_S10_F274', 'L0_S11_F282', 'L0_S11_F286', 'L0_S11_F290', 'L0_S11_F294

In [58]:
# take a subset of data
mldf=df.limit(50)
mldf.describe().show(3)

+-------+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+---------+-------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------------+--------------------+-------------------+--------------------+--------------------+-------------------+--------------------+--------------------+----------+--------------------+-------------------+--------------------+--------------------+-----------

In [52]:
mldf.na.drop().take(3)

[]

In [28]:
#features=mldf.select([c for c in mldf.columns if c in featureColumnNames])
#labels=mldf.select(["Response"])

from pyspark.ml.feature import VectorAssembler
#vectorAssembler = VectorAssembler(inputCols = featureColumnNames, outputCol = 'features')
vectorAssembler = VectorAssembler(inputCols = ['L0_S0_F14', 'L0_S0_F18'], outputCol = 'features')
prepped_df = vectorAssembler.transform(mldf)
prepped_df = prepped_df.select(['features', 'Response'])


In [29]:
prepped_df.show(3)

An error occurred while calling o379.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 16.0 failed 4 times, most recent failure: Lost task 0.3 in stage 16.0 (TID 170, ip-172-31-35-9.eu-west-1.compute.internal, executor 1): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$3: (struct<L0_S0_F14:double,L0_S0_F18:double>) => vector)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage2.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$10$$anon$1.hasNext(WholeStageCodegenExec.scala:614)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:253)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:247)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartitionsInternal$1$$anonfun$apply$25.apply(RDD.s

In [ ]:
splits = prepped_df.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]

In [9]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol='Response', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_df)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

name 'train_df' is not defined
Traceback (most recent call last):
NameError: name 'train_df' is not defined

